Title: Assignment: 9.1
Author: Sarah Yawn
Date: 2 Aug 2025
Modified By: Sarah Yawn
Description:
Data: kagglehub.dataset_download("nelgiriyewithana/emotions")

In [6]:
#Install Packages
# !pip install transformers>=4.11.3
# !pip install accelerate>=0.21.0
# !pip install "transformers[torch]>=4.11.3"
# !pip install datasets
#!pip install sacrebleu
#!pip install evaluate

## Restart Kernel

In [7]:
#Install Packages
# !pip install transformers>=4.11.3
# !pip install accelerate>=0.21.0
# !pip install "transformers[torch]>=4.11.3"
# !pip install datasets
#!pip install sacrebleu
#!pip install evaluate

## Restart Kernel
#!pip install evaluate sacrebleu

In [8]:
# Import necessary libraries
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

import numpy as np
import kagglehub
import os
import pandas as pd

import evaluate

##Dataset Selection:
Identify and select a dataset that aligns with the text generation application you wish to focus on. Ensure the dataset is suitable for NLP tasks and preprocess it accordingly.

In [9]:
# Download the dataset and get the path to the directory
dataset_dir = kagglehub.dataset_download("nelgiriyewithana/emotions")
csv_path = os.path.join(dataset_dir, 'text.csv')

# Read the CSV and extract the 'text' column
df = pd.read_csv(csv_path)
text_data = df['text'].sample(1).tolist()

# Write the text data to the combined dataset file
combined_dataset_path = 'combined_dataset.txt'
with open(combined_dataset_path, 'w') as outfile:
    for item in text_data:
        outfile.write("%s\n" % item)

print(f"Text data from the CSV has been written to: {combined_dataset_path}")

100%|██████████| 15.7M/15.7M [00:00<00:00, 102MB/s]

Extracting files...


Text data from the CSV has been written to: combined_dataset.txt


##Model Preparation:
Choose a pre-trained model from the Hugging Face Model Hub that best fits your chosen application. Initialize this model along with its tokenizer.

In [10]:
# Load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2-medium'  # Change to the desired pre-trained model size
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
# Inspect the dataset directory and the combined file
import os

dataset_dir = kagglehub.dataset_download("nelgiriyewithana/emotions")

print("Contents of the dataset directory:")
for filename in os.listdir(dataset_dir):
    print(f"- {filename}")

combined_dataset_path = 'combined_dataset.txt'
if os.path.exists(combined_dataset_path):
    with open(combined_dataset_path, 'r') as f:
        print("\nFirst 5 lines of the combined dataset file:")
        for i in range(5):
            line = f.readline()
            if not line:
                break
            print(line.strip())
else:
    print("\ncombined_dataset.txt does not exist.")

Using Colab cache for faster access to the 'emotions' dataset.
Contents of the dataset directory:
- .nfs0000000034c43a5d00000025
- text.csv

First 5 lines of the combined dataset file:
i feel so honored to be chosen to raise you


In [12]:
from datasets import load_dataset

# Load the dataset from the combined text file
train_dataset = load_dataset('text', data_files={'train': 'combined_dataset.txt'})['train']

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=['text'])

Generating train split: 0 examples [00:00, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [13]:
# Define training arguments and trainer
training_args = TrainingArguments(
    output_dir="llm-fine-tuned",  # Directory to save the fine-tuned model and logs
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size per GPU/CPU during training
    save_steps=1000,  # Save model checkpoint every specified number of steps
    save_total_limit=2,  # Limit the total number of saved checkpoints
    prediction_loss_only=False,  # Only compute the prediction loss
)

In [14]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    ),
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

##Model Fine-Tuning:
Customize the training parameters such as learning rate, batch size, and number of epochs to optimize your model's performance for the specific type of text you're working with.

api key: f8f9072b13fa33262c3623bed48b1ce7849e59e0

In [15]:
# Fine-tune the pre-trained model on your dataset
trainer.train()
trainer.save_model("llm-fine-tuned")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sarahkateyawn (sarahkateyawn-bellvue-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


I am having a hard time getting this program to run the fine tuning without overusing the RAM or hitting a timeout error. If this continues to be an issue, you may want to skip to the practical applications block and run from there. I did however, limit the input file to select only one piece of text, which will hopefully help in the future.

In [16]:
# Load fine-tuned model for testing
fine_tuned_model = GPT2LMHeadModel.from_pretrained("llm-fine-tuned")

# Define a prompt or input text for generation
prompt = "In recent years, artificial intelligence has revolutionized"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate text based on the prompt
output = fine_tuned_model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7, do_sample=True)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text:
In recent years, artificial intelligence has revolutionized the way we learn. It has revolutionized our ability to learn. When we want to learn, we choose to use technology. When we want to learn, we use artificial intelligence. Today, we are taking the first step toward a world where we all share the same dreams of learning and sharing experience and learning together.

We are entering a new era of learning, and we have begun to see the first signs of it. I just finished working


##Practical Applications:
Develop practical text generation tasks that utilize your fine-tuned model. This could involve generating creative text, automating customer service responses, or providing analytical summaries.

In [17]:
# Practical Applications

# 1. Creative Text Generation
prompt_creative = "Once upon a time, in a land filled with mystical creatures,"
input_ids_creative = tokenizer.encode(prompt_creative, return_tensors="pt")
output_creative = fine_tuned_model.generate(
    input_ids_creative,
    max_length=150,
    num_return_sequences=1,
    temperature=0.8,
    do_sample=True
)
generated_text_creative = tokenizer.decode(output_creative[0], skip_special_tokens=True)
print("Generated Creative Text:")
print(generated_text_creative)

# 2. Customer Service Response
prompt_customer_service = "A customer is angry about a delayed shipment. Write a polite and helpful response."
input_ids_customer_service = tokenizer.encode(prompt_customer_service, return_tensors="pt")
output_customer_service = fine_tuned_model.generate(
    input_ids_customer_service,
    max_length=100,
    num_return_sequences=1,
    temperature=0.7,
    do_sample=True
)
generated_text_customer_service = tokenizer.decode(output_customer_service[0], skip_special_tokens=True)
print("\nGenerated Customer Service Response:")
print(generated_text_customer_service)

# 3. Analytical Summary
prompt_summary = "Summarize the key benefits of using renewable energy sources."
input_ids_summary = tokenizer.encode(prompt_summary, return_tensors="pt")
output_summary = fine_tuned_model.generate(
    input_ids_summary,
    max_length=120,
    num_return_sequences=1,
    temperature=0.6,
    do_sample=True
)
generated_text_summary = tokenizer.decode(output_summary[0], skip_special_tokens=True)
print("\nGenerated Analytical Summary:")
print(generated_text_summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Creative Text:
Once upon a time, in a land filled with mystical creatures, we used to gather under the banner of the Light and speak to them in ways that only you could understand. Perhaps it was to teach them of the ways of this world and that of the other." — The Book of the Stranger, Anueth, The Guardian

The ancient, yet mysterious, language of the gods, spoken by ancient beings whose hearts were not human, was not only the language of the living. The gods and the other immortal beings spoke an ancient language, but it came from another world, which they called the Source of All. That world was not simply a place to meet. It was a place of secrets and mysteries that only the most revered of beings


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Customer Service Response:
A customer is angry about a delayed shipment. Write a polite and helpful response.

Your customer needs to feel heard and respected.

Give your customer a choice to choose a more appealing price.

Write a customer's address and phone number.

Your customer needs to be proud of your service.

Ask for help on a request.

Write a customer's name, address, email address, and phone number.


Your customer needs to feel like they are

Generated Analytical Summary:
Summarize the key benefits of using renewable energy sources.

A key advantage of using renewable energy sources (renewables) is the fact that you can save money on your energy bills.

Because of this, it is very important to determine the best choice for your household.

For example, if you have an electric car, you should consider buying a solar panel to reduce your monthly electricity bill.

If you are considering buying a solar energy system, you should also consider buying a solar energy s

This is pretty impressive, obviously it is fairly clunky "magical magic" and "Try to help the customer understand what he or she is going through." are not necissarily beneficial and definetely need to be looked at and reevaluated before being used in a production environment, but, still an impressive achievement.

##Model Evaluation:
Evaluate your model using metrics suitable for text generation, such as perplexity or BLEU score, to understand its effectiveness and areas for improvement.

In [18]:
!pip install evaluate sacrebleu

In [19]:
# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("llm-fine-tuned")

# 1. Perplexity
eval_results = trainer.evaluate()
perplexity = np.exp(eval_results['eval_loss'])
print(f"Perplexity: {perplexity}")

# 2. BLEU Score
bleu = evaluate.load("sacrebleu")

# Generate predictions for the evaluation dataset
predictions = []
references = []
for example in tokenized_dataset:
    input_ids = torch.tensor(example['input_ids']).unsqueeze(0)
    output = fine_tuned_model.generate(input_ids, max_length=50, num_return_sequences=1)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(decoded_output)
    references.append([tokenizer.decode(example['input_ids'], skip_special_tokens=True)])


# Compute BLEU score
bleu_score = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {bleu_score['score']}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Perplexity: 2.0056017299485513


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


BLEU Score: 18.940208900425674


##Iterative Improvement:
Refine your model based on performance feedback. Experiment with different configurations and training techniques to enhance its accuracy and response quality.

In [20]:
# 1. Hyperparameter Tuning
# You can create a new TrainingArguments object with different hyperparameters
training_args_tuned = TrainingArguments(
    output_dir="llm-fine-tuned-tuned",
    overwrite_output_dir=True,
    num_train_epochs=5,  # Increase the number of epochs
    per_device_train_batch_size=4,  # Decrease the batch size
    learning_rate=5e-5,  # Adjust the learning rate
    save_steps=2000,
    save_total_limit=3,
    prediction_loss_only=True,
)

# Initialize a new trainer with the tuned arguments
trainer_tuned = Trainer(
    model=model,
    args=training_args_tuned,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    ),
    train_dataset=tokenized_dataset,
)

# Train the model with the new hyperparameters
trainer_tuned.train()

Step,Training Loss


TrainOutput(global_step=5, training_loss=0.40222859382629395, metrics={'train_runtime': 586.4028, 'train_samples_per_second': 0.009, 'train_steps_per_second': 0.009, 'total_flos': 90693427200.0, 'train_loss': 0.40222859382629395, 'epoch': 5.0})

In [ ]:
# 2. Different Model Size
# You can load a different pre-trained model from the Hugging Face Model Hub
# For example, to use 'gpt2-large':
tokenizer_large = GPT2Tokenizer.from_pretrained('gpt2-large')
model_large = GPT2LMHeadModel.from_pretrained('gpt2-large')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

In [ ]:
# 3. Data Augmentation (Example with back-translation)
# This is a more advanced technique and requires a translation model.
# Here is a conceptual example:
from transformers import MarianMTModel, MarianTokenizer

# # Load a translation model (e.g., English to German and back to English)
en_to_de_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')
en_to_de_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-de')
de_to_en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-de-en')
de_to_en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-de-en')

def augment_text(text):
    # Translate to German
    translated_to_de = en_to_de_model.generate(**en_to_de_tokenizer(text, return_tensors="pt", padding=True))
    text_in_de = [en_to_de_tokenizer.decode(t, skip_special_tokens=True) for t in translated_to_de]

    # Translate back to English
    translated_to_en = de_to_en_model.generate(**de_to_en_tokenizer(text_in_de[0], return_tensors="pt", padding=True))
    augmented_text = [de_to_en_tokenizer.decode(t, skip_special_tokens=True) for t in translated_to_en]

    return augmented_text[0]

# # Example usage:
original_text = "This is an example of data augmentation."
augmented_text = augment_text(original_text)
print(f"Original: {original_text}")
print(f"Augmented: {augmented_text}")

I have been crashing alot on the iterative improvement step due to RAM overuse, though these have run with relative success previously, it seems that the fine tuning step and the iterative improvement step cannot run in the same session without over consuming the RAM. The problem is this causes a sesion failure, so I have to restart from the top which is not ideal. Overall, I am not terribly sure how to fix this without purchasing a premium version of Collab.